# RFE Analysis

In [4]:
%reset -f
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import resample, butter, filtfilt

from mat73 import loadmat

from util.kinematics import import_vicon_lbi
from util.tendontapping import calculate_wavespeed

%matplotlib qt

## Import data

In [5]:
participant = 'MK'
condition = 'Fifty_070_1'
points = pd.read_csv(f'../data/raw/kinematics/{participant}/{condition}.csv',
                     sep=',', skiprows=6, header=0,
                     usecols=[x for x in range(2, 29)])
spike = loadmat(f'../data/raw/spike/{participant}/{condition}.mat', use_attrdict=True)

## Sync with Motive, retrieve relevant spike data

In [6]:
sync_start = list(map(lambda i: i > 0.6, spike.Motive['values'])).index(True)
print(sync_start)
try:
    sync_stop = list(map(lambda i: i < 2,
                     spike.Motive['values'][sync_start+200:])).index(True) + sync_start + 200
except:
    sync_stop = -1

if sync_stop == -1:
    tq_len = len(spike.Torque['values']) - sync_start
    Tq = spike.Torque['values'][sync_start:]
    Angle = spike.Angle['values'][sync_start:]
    Emg = spike.VL['values'][sync_start:] + spike.VM['values'][sync_start:] + spike.RF['values'][sync_start:]
else:
    tq_len = sync_stop - sync_start
    Tq = spike.Torque['values'][sync_start:sync_stop]
    Angle = spike.Angle['values'][sync_start:sync_stop]
    Emg = spike.VL['values'][sync_start:sync_stop] + spike.VM['values'][sync_start:sync_stop] + spike.RF['values'][sync_start:sync_stop]

3456


## Filter data

In [7]:
b, a = butter(4, 20, btype='low', fs=2000)
Tq = filtfilt(b, a, Tq)
Angle = filtfilt(b, a, Angle)

Emg_rms = np.convolve(Emg**2, np.ones(500)/500, mode='same')**0.5


#first_tap = list(map(lambda i: i > 0.6, spike.TT['values'])).index(True) / 2000
#z = np.arange(first_tap, first_tap+14.98, 0.02)
ws = calculate_wavespeed(spike.Acc1['values'], spike.Acc2['values'],
                         spike.DigMark['times'])

ws_sq = ws ** 2

## Label markers

In [8]:
Lml, Mml, Kjc, Thigh, Shnk_top, Shnk_bot, Shnk_mid, Pad, Axs, Rot, Crnk = import_vicon_lbi(points)

# plot of all markers
markers = [Lml, Mml, Kjc, Thigh, Shnk_top, Shnk_bot, Shnk_mid, Pad, Axs, Rot, Crnk]
markers_list = ['Lml', 'Mml', 'Kjc', 'Thigh', 'Shnk_top', 'Shnk_bot', 'Shnk_mid', 'Pad', 'Axs', 'Rot', 'Crnk']
fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')
for c in markers:
    ax1.plot(c[:, 0], c[:, 1], c[:, 2])
fig.legend(markers_list)

## Calculate angles

In [9]:
# pad plane
u_pad = Shnk_top - Shnk_bot
v_pad = Shnk_mid - Shnk_bot
n_pad = np.cross(u_pad, v_pad)
n_pad_n = n_pad / np.linalg.norm(n_pad, axis=1)[:, None]

# isomed plane
u_iso = Rot - Crnk
v_iso = Axs - Crnk
n_iso = np.cross(u_iso, v_iso)
n_iso_n = n_iso / np.linalg.norm(n_iso, axis=1)[:, None]

# thigh plane
u_thigh = Lml - Mml
v_thigh = Thigh - Mml
n_thigh = np.cross(u_thigh, v_thigh)
n_thigh_n = n_thigh / np.linalg.norm(n_thigh, axis=1)[:, None]

# angle in radians
angle_pad_iso = np.arccos(np.sum(n_pad_n * n_iso_n, axis=1))
angle_pad_thigh = np.arccos(np.sum(n_pad_n * n_thigh_n, axis=1))

# upsampled angle, knee angle in degrees
angle_pad_iso_tq = resample(angle_pad_iso, tq_len)
angle_pad_thigh_tq = resample(angle_pad_thigh, tq_len)
knee_angle = 180 - (angle_pad_thigh_tq *180/np.pi)


## Calculate corrected torque

In [28]:
# find point P on isomed lever arm that is shortest distance from the axis
d = (Shnk_mid[0, :] - Pad[0, :]) / np.linalg.norm(Shnk_mid[0, :] - Pad[0, :])
v = Axs[0, :] - Pad[0, :]
t = np.dot(v, d)
P = Pad[0, :] + np.dot(t, d)

# lever arm as the distance between P and the axis, corrected lever arm between KJC and center of the Pad
La = np.linalg.norm(P-Axs[0, :])
cLa = np.linalg.norm(Kjc-Pad, axis=1)
# if kinemetic data longer than torque data cut it off at the end
if sync_stop == -1:
    cLa = cLa[:int(tq_len / 20)]
# upsample kinemetic lever arm data such that corrected torque can be calculated
cLa = resample(cLa, tq_len)

# force on the isomed based on the lever arms and the angle between the force application
Fiso = Tq / La
Fpad = Fiso / abs(np.cos(angle_pad_iso_tq))
crTq = Fpad * cLa

## Fitfy percent reference contractions

In [29]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax1.plot(knee_angle)
ax1.vlines(7.5*2000-sync_start, np.min(knee_angle), np.max(knee_angle))
ax3.plot(Tq)
ax3.plot(crTq)
ax3.vlines(7.5*2000-sync_start, np.min(crTq), np.max(crTq), 'red')
ax2.plot(ws)
ax4.plot(ws**2)
ax4.vlines(6.5*50, np.min(ws**2), np.max(ws**2), 'red')
bases = plt.ginput(2, timeout=-1)
tq_base = int(bases[0][0])
ws_base= int(bases[1][0])
%matplotlib inline

start = int(7.5*2000-sync_start)
stop = int(8.5*2000-sync_start)
ka_ss = np.mean(knee_angle[start:stop])
crtq_ss = np.mean(crTq[start:stop]) - np.mean(crTq[tq_base:tq_base+int(0.5*2000)])
ws_ss = np.mean(ws[int(6.5*50):int(7.5*50)]) - np.mean(ws[ws_base:ws_base+int(0.5*50)])
ws_sq_ss = np.mean(ws_sq[int(6.5*50):int(7.5*50)]) - np.mean(ws_sq[ws_base:ws_base+int(0.5*50)])

results = pd.DataFrame([crtq_ss, None, ws_ss, ws_sq_ss, None, ka_ss])
results.to_clipboard(index=False)


## RFE conditions

In [32]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax1.plot(knee_angle)
ax1.plot(Angle)
ax3.plot(Tq)
ax3.plot(crTq)
ax2.plot(ws)
ax4.plot(ws**2)
bases = plt.ginput(2, timeout=-1)
rfe_base = int(bases[0][0])
ws_base= int(bases[1][0])
%matplotlib inline

rfe_time = 18633 - sync_start
ws_ss_time = int(18633/2000*50)

ws_ss = np.mean(ws[ws_ss_time:ws_ss_time+int(0.5*50)]) - np.mean(ws[ws_base:ws_base+int(0.5*50)])
ws_sq_ss = np.mean(ws_sq[ws_ss_time:ws_ss_time+int(0.5*50)]) - np.mean(ws_sq[ws_base:ws_base+int(0.5*50)])

tq_ss = np.mean(Tq[rfe_time:rfe_time+int(0.5*2000)]) - np.mean(Tq[rfe_base:rfe_base+int(0.01*2000)])
ctq_ss = np.mean(crTq[rfe_time:rfe_time+int(0.5*2000)]) - np.mean(crTq[rfe_base:rfe_base+int(0.01*2000)])

k_angle_ss = np.mean(knee_angle[rfe_time:rfe_time+int(0.5*2000)])
i_angle_ss = np.mean(Angle[rfe_time:rfe_time+int(0.5*2000)])
k_angle_rest = np.mean(knee_angle[rfe_base:int(rfe_base+int(0.01*2000))])
i_angle_rest = np.mean(Angle[rfe_base:int(rfe_base+int(0.01*2000))])
k_angle_bef = np.min(knee_angle)
i_angle_bef = np.min(Angle)

results = pd.DataFrame([tq_ss, ctq_ss, None, ws_ss, ws_sq_ss, None, k_angle_rest, k_angle_bef, k_angle_ss, None, i_angle_rest, i_angle_bef, i_angle_ss])
results.to_clipboard(index=False)
